# Singular Value Decomposition
covers matrix decompsition and how can a matrix be ranked deficient. and how we can produce smaller matrid that captures most of the information.

### **What is a Rank-Deficient Matrix?**  
A rank-deficient matrix is a matrix where the rank (the number of linearly independent rows or columns) is less than its full possible rank. For an \( m \times n \) matrix, the rank is at most \( \min(m, n) \). If the rank is less than this, some rows or columns are linearly dependent, meaning they can be expressed as linear combinations of others.

---

### **Why is \( W \) Rank Deficient?**  
In the context of pre-trained models:
- **High Dimensionality but Low Complexity**: The weight matrix \( W \) often has very large dimensions (e.g., 1000 × 1000), but the information it encodes is structured and concentrated in a much smaller subspace.  
- **Intrinsic Low Rank**: Empirical studies show that the task-specific updates or features learned by \( W \) during training are inherently low-dimensional, meaning only a small number of linearly independent components are sufficient to capture most of the useful information.  

Thus, \( W \) is rank deficient because many of its rows or columns are redundant or dependent.

In [5]:
import torch
import numpy as np

_ = torch.manual_seed(0)

In [6]:
#  generate a rank deficient matrix W
d, k = 10, 10

# this way we can gernate a rank deficicent matrix

W_rank = 2
W = torch.randn(d, W_rank) @ torch.randn(W_rank, k)
print(W)

tensor([[-1.0797,  0.5545,  0.8058, -0.7140, -0.1518,  1.0773,  2.3690,  0.8486,
         -1.1825, -3.2632],
        [-0.3303,  0.2283,  0.4145, -0.1924, -0.0215,  0.3276,  0.7926,  0.2233,
         -0.3422, -0.9614],
        [-0.5256,  0.9864,  2.4447, -0.0290,  0.2305,  0.5000,  1.9831, -0.0311,
         -0.3369, -1.1376],
        [ 0.7900, -1.1336, -2.6746,  0.1988, -0.1982, -0.7634, -2.5763, -0.1696,
          0.6227,  1.9294],
        [ 0.1258,  0.1458,  0.5090,  0.1768,  0.1071, -0.1327, -0.0323, -0.2294,
          0.2079,  0.5128],
        [ 0.7697,  0.0050,  0.5725,  0.6870,  0.2783, -0.7818, -1.2253, -0.8533,
          0.9765,  2.5786],
        [ 1.4157, -0.7814, -1.2121,  0.9120,  0.1760, -1.4108, -3.1692, -1.0791,
          1.5325,  4.2447],
        [-0.0119,  0.6050,  1.7245,  0.2584,  0.2528, -0.0086,  0.7198, -0.3620,
          0.1865,  0.3410],
        [ 1.0485, -0.6394, -1.0715,  0.6485,  0.1046, -1.0427, -2.4174, -0.7615,
          1.1147,  3.1054],
        [ 0.9088,  

In [4]:
# evaluatin rank
W_rank = np.linalg.matrix_rank(W)
print(f"{W_rank=}")

W_rank=2


## perform SVD on the W matrix



In [10]:
## perform SVD of the W matrix.. which produces three matrices U, S and V where W = UxSxV^T but dims of U,S and V are much smaller
U, S, V = torch.svd(W)
W.shape, U.shape, S.shape, V.shape

(torch.Size([10, 10]),
 torch.Size([10, 10]),
 torch.Size([10]),
 torch.Size([10, 10]))

# Matrix Rank

In the context of **LoRA** and its explanation, **rank** refers to the **matrix rank**, not the order or sequence.

---

### **What is Matrix Rank?**

The rank of a matrix is the maximum number of linearly independent rows or columns in the matrix. It is a measure of the **complexity** or **dimensionality** of the space that the matrix spans.

For example:
1. A \( 3 \times 3 \) matrix:
   - If all three rows (or columns) are linearly independent, the rank is 3.
   - If only two rows (or columns) are linearly independent, the rank is 2.
   - If only one row (or column) is linearly independent, the rank is 1.

Mathematically:
- For a matrix \( A \in \mathbb{R}^{m \times n} \), the rank \( r \) is:
  \[
  r = \text{dim}(\text{span of rows}) = \text{dim}(\text{span of columns})
  \]

---

### **Rank in LoRA**

In LoRA, the rank refers to the rank \( r \) of the **low-rank decomposition** of the task-specific update matrix \( \Delta W \).

\[
\Delta W = A B^\top
\]

Here:
- \( A \in \mathbb{R}^{d_{\text{out}} \times r} \)
- \( B \in \mathbb{R}^{d_{\text{in}} \times r} \)

The rank \( r \) is much smaller than \( \min(d_{\text{out}}, d_{\text{in}}) \), meaning \( \Delta W \) can be represented using far fewer independent components. This low rank indicates that the adjustment needed to fine-tune the model is inherently simple and does not require a full update of the large weight matrix \( W \).

---

### **Why is Low Rank Important?**

1. **Efficiency**: 
   - A low-rank matrix requires fewer parameters to represent, reducing memory and computational costs.
   - For example, if \( d_{\text{out}} = 10,000 \), \( d_{\text{in}} = 10,000 \), and \( r = 4 \):
     - Full matrix: \( 10,000 \times 10,000 = 100 \, \text{million parameters} \)
     - Low-rank update: \( 10,000 \times 4 + 10,000 \times 4 = 80,000 \, \text{parameters} \)

2. **Interpretability**:
   - A low rank suggests that only a small number of key dimensions (or directions) need adjustment to adapt the model to a specific task.
   - This highlights the structured nature of how LLMs encode knowledge.

---

In summary, **rank in LoRA refers to the matrix rank** of the task-specific update matrix \( \Delta W \), representing the dimensionality of the adjustment space. It’s a critical factor in achieving efficient and targeted fine-tuning of large language models.